# Implementation of common string model for MCSP

In [1]:
from typing import *

import numpy as np
import random

from ortools.init import pywrapinit
from ortools.linear_solver import pywraplp

## Helper functions

In [2]:
def gen_substring_set(s: str) -> Set[str]:
  """This function returns all substring of the given string s

  Args:
      s (str): a string we want extract all substring

  Returns:
      Set[str]: A set containing every substring without duplicates.
  """
  res: Set[str] = set()

  for i in range(len(s)+1):
    for j in range(i+1,len(s)+1):
      res.add(s[i:j])

  return res

In [3]:
def gen_substring_pos(s: str, T: Set[str]) -> Dict[str, List[Tuple[int,int]]]:
  """This function returns dictionary which every key is a substring of 's' 
     and it is in T, ie. it is a common substring of S1 and S2 input strings

  Args:
      s (str): a input string
      T (Set[str]): All substrings in common of S1 and S2

  Returns:
      Dict[str, List[Tuple[int,int]]]: All common substring occurrences indexes
  """
  res: Dict[str, List[Tuple[int,int]]] = {}

  for i in range(len(s)+1):
    for j in range(i+1,len(s)+1):
      if s[i:j] in T:
        if s[i:j] in res:
          res[s[i:j]].append((i,j))
        else:
          res[s[i:j]] = [(i,j)]

  return res

In [4]:
def shufle_string(s):
  l = list(s)
  random.shuffle(l)
  result = ''.join(l)
  return result

In [5]:
# S1 = 'AGACTG'
# S2 = 'ACTAGG'
# res: 3

#S1 = '11234'*50
#S2 = shufle_string(S1)

S1 = '11431131222222242211241142233142424314334333334114212232341142441422241314343143441412214343314324312341333414223221242432233112313443421344143312213144413121'
S2 = '21114222233233431242212421223443444222434144334233243143431241134141124112331441131423434214333342211141234323241224133411332243342213211142413134241311424312'
# res: 52


N = len(S1)

In [6]:
SUBS_OF_S1 = gen_substring_set(S1)
SUBS_OF_S2 = gen_substring_set(S2)

In [7]:
T = set.intersection(SUBS_OF_S1, SUBS_OF_S2)

In [8]:
Q = [gen_substring_pos(S1, T), gen_substring_pos(S2, T)]

In [9]:
#T

In [10]:
#Q[0]

In [11]:
#Q[1]

## Chosing the solver

In [12]:
# solver = pywraplp.Solver.CreateSolver('BOP_INTEGER_PROGRAMMING')
solver = pywraplp.Solver.CreateSolver('CBC')

# Variables

In [13]:
y = {}
for t_idx, t in enumerate(T):
  for q_idx,q in enumerate(Q):
    for k in q[t]:
      y[t_idx,q_idx,k[0],k[1]] = solver.BoolVar(f'y[{q_idx},{t_idx},{k[0]},{k[1]}]')

In [14]:
print('Number of variables =', solver.NumVariables())

Number of variables = 1172


# Constraints

caso: $\sum\limits_{k \in Q^{1}_{t}} y^{1}_{t,k} = \sum\limits_{k \in Q^{2}_{t}} y^{2}_{t,k}, \forall t \in T$

This restrictions ensures that the number of partitions of both input strings are equal.

In [15]:
for t_idx, t in enumerate(T):
  partitions_of_S1 = [ y[t_idx, 0, k[0], k[1]] for k in Q[0][t] ]
  partitions_of_S2 = [ y[t_idx, 1, k[0], k[1]] for k in Q[1][t] ]

  solver.Add(solver.Sum(partitions_of_S1) == solver.Sum(partitions_of_S2))

caso: $\sum\limits_{t \in T} \sum\limits_{k \in Q^{1}_{t} | k \leq j < k + |t| } y^{1}_{t,k} = 1, \forall j = 1, \dots, n$

Ensures that only one subtring at position $j$ in the $S_1$ was chosen.

In [16]:
for j in range(N):
  substrings_at_pos_j_of_S1 = []
  for t_idx, t in enumerate(T):
    for k in Q[0][t]: 
      if k[0] <= j < k[1]: # k[1] == k[0] + size(t)
        substrings_at_pos_j_of_S1.append(y[t_idx,0,k[0],k[1]])
  
  solver.Add(solver.Sum(substrings_at_pos_j_of_S1) == 1)

caso: $\sum\limits_{t \in T} \sum\limits_{k \in Q^{2}_{t} | k \leq j < k + |t| } y^{2}_{t,k} = 1, \forall j = 1, \dots, n$

Ensures that only one substring at position $j$ in the $S_2$ was chosen.

In [17]:
for j in range(N):
  substrings_at_pos_j_of_S2 = []
  for t_idx, t in enumerate(T):
    for k in Q[1][t]:
      if k[0] <= j < k[1]: # k[1] == k[0] + size(t)
        substrings_at_pos_j_of_S2.append(y[t_idx,1,k[0],k[1]])

  solver.Add(solver.Sum(substrings_at_pos_j_of_S2) == 1)

# Objective Function

função: $\text{Minimize} \sum\limits_{t \in T} \sum\limits_{k \in Q^{1}_{t}} y^{1}_{t,k}$

In [18]:
objective_terms = []
for t_idx, t in enumerate(T):
  for k in Q[0][t]:
    objective_terms.append(y[t_idx,0,k[0],k[1]])

solver.Minimize(solver.Sum(objective_terms))

# Solving

In [19]:
status = solver.Solve()

In [20]:
if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
  print(solver.Objective().Value())
else:
  print("There is no optimal solution.")

52.0


In [21]:
res = []
for t_idx, t in enumerate(T):
  for k in Q[0][t]:

    if abs(y[t_idx,0,k[0],k[1]].solution_value() - 1) <= 0.01:
      #print(f'{t_idx},{0},{k[0]},{k[1]}: {y[t_idx,0,k[0],k[1]].solution_value()}')
      res.append(S1[k[0]:k[1]])

In [22]:
res.sort()

In [23]:
len(res)

52

In [24]:
#res

In [25]:
# with open('modelo.txt', 'w') as f:
#   f.write(solver.ExportModelAsLpFormat(False))

In [26]:
N

158